In [237]:
import pandas as pd
import plotly.express as px

## Calificar examen estudiantes: Dimensión 7

In [238]:
est = pd.read_excel('../../../data/raw/Est_Con_Pre_020522.xlsx')
print(est.shape)
est.columns

FileNotFoundError: [Errno 2] No such file or directory: '../../../data/raw/Est_Con_Pre_020522.xlsx'

In [66]:
est['Timestamp'] = pd.to_datetime(est['Timestamp'])
est = est[est.Timestamp>'2022-04-14']
est['Fecha'] = est.Timestamp.dt.strftime('%d/%m')
est['Fecha'][:5]

0    19/04
1    19/04
2    19/04
3    19/04
4    19/04
Name: Fecha, dtype: object

In [67]:
est = est.drop(columns=est.filter(regex=r'eliminar').columns)
est = est.drop(columns='Timestamp')
est = est.dropna(subset='Número de lista')
est = est[est['Código IE']<300]
est.shape

(5082, 71)

In [68]:
respuestas_correctas = {
    "Un algoritmo es:":"Una secuencia lógica de pasos para realizar una tarea",
    "¿Para qué sirven los algoritmos?":"Para planificar la solución a un problema",
    "Un bucle es:":"Un conjunto de instrucciones que se ejecuta mientras se cumpla una condición",
    "¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?":"a.",
    "¿Qué mensaje deseaba enviar la líder Wayuú?":"c. Nublado",
    "¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?":"a.",
    "¿Cuál será la foto con más vistas?":"c) Julio",
    "Ayuda al robot verde a salir del laberinto":"b.",
    "Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?":"c) Si Óscar empaca Deditos para merendar, puede hacer Arroz de pollo para almorzar",
    "¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?":"a.",
    "¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?":"a) La botella B debe ser verde"}

In [69]:
est.columns.tolist()

['Número de lista',
 'Edad',
 'Sexo',
 'Sector vivienda',
 'Internet',
 'Uso del dispositivo móvil',
 'Nivel escolaridad madre',
 'Nivel escolaridad padre',
 'Ocupación madre',
 'Ocupación padre',
 '¿Con quién vives?',
 'Grado',
 '1.1. Ingeniería',
 '2.1 Soy capaz de sacar buenas notas en esta asignatura',
 '2.2 Si me va bien en esta asignatura, me ayudará en mi futura ocupación',
 '2.3 A mis padres les gustaría que eligiera un futuro profesional relacionado a esta asignatura',
 '2.4 Sé de alguien en mi familia que utiliza conocimientos relacionados a esta asignatura en su ocupación',
 'comentarios 1-2',
 'Un algoritmo es:',
 '¿Para qué sirven los algoritmos?',
 'Un bucle es:',
 '3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional',
 '3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional',
 '3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional',
 '3.4 Siento que puedo expli

In [136]:
col_index = ['Edad', 'Sexo', 'Código IE', '¿Te reconoces como una persona con algún tipo de discapacidad?']
col_conocePC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[0]
col_PC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[1:5]
col_autoeficacia = est.filter(regex=r'^3.*', axis=1).columns.tolist()[-5:]
col_interes = est.filter(regex=r'^2.*', axis=1).columns.tolist()
col_conocimiento = list(respuestas_correctas.keys())
col_PC

['3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional',
 '3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional',
 '3.4 Siento que puedo explicar la forma en que las sub-habilidades del pensamiento computacional se correlacionan con la programación',
 '3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar']

In [71]:
est = est.reset_index()

In [156]:
df_interes = est[['index']+col_interes].melt(id_vars='index', value_name='Seleccion', var_name='Pregunta')
df_interes['Interes en tecnologia'] = 1*(df_interes['Seleccion'].str.contains('Tecnología'))
df_puntaje_interes = df_interes.pivot_table(index='index', values=['Interes en tecnologia'], aggfunc=['sum', 'max']).reset_index()
df_puntaje_interes.columns = ['index', 'Puntaje interes', 'Interesado en tecnologia']
df_puntaje_interes['Puntaje interes'] = 100*(df_puntaje_interes['Puntaje interes']/4)
#df_puntaje_interes = (100*df_interes.groupby('index')['Interes en tecnologia'].sum()/4).reset_index()
df_puntaje_interes

,index,Puntaje interes,Interesado en tecnologia
0,1,100.0,1
1,2,75.0,1
2,3,25.0,1
3,4,100.0,1
4,5,25.0,1
...,...,...,...
5077,5253,100.0,1
5078,5254,25.0,1
5079,5255,25.0,1
5080,5256,50.0,1


In [157]:

df_conocimientos = est[['index']+col_conocimiento].melt(id_vars='index', value_name='Respuesta estudiante', var_name='Pregunta')
df_conocimientos['Respuesta correcta'] = df_conocimientos['Pregunta'].replace(respuestas_correctas)
df_conocimientos['Puntaje conocimiento'] = 1*(df_conocimientos['Respuesta estudiante'] == df_conocimientos['Respuesta correcta'])
df_puntaje = (df_conocimientos.groupby('index')['Puntaje conocimiento'].sum()*100/11).reset_index()
df_puntaje.describe()

,index,Puntaje conocimiento
count,5082.000000,5082.000000
mean,2619.399646,31.195664
std,1510.114305,19.302506
min,1.000000,0.000000
25%,1324.250000,18.181818
50%,2610.500000,27.272727
75%,3934.750000,45.454545
max,5257.000000,100.000000


In [158]:
df_7 = pd.merge(est, df_puntaje, on='index')
df_7 = pd.merge(df_7, df_puntaje_interes, on='index')
df_7

,index,Número de lista,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,...,Tipo de discapacidad,¿Te reconoces como una persona con algún tipo de discapacidad?,1.8 Desarrollo de software,Conoce GreenTIC,Código IE,Grupo,Fecha,Puntaje conocimiento,Puntaje interes,Interesado en tecnologia
0,1,3.0,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,...,NaN,No,Me interesa poco,No,166.0,B o 02,19/04,45.454545,100.0,1
1,2,7.0,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,...,NaN,No,Está entre mis preferidas,No,166.0,B o 02,19/04,18.181818,75.0,1
2,3,30.0,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,...,NaN,No,Me interesa poco,No,166.0,B o 02,19/04,36.363636,25.0,1
3,4,6.0,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,...,NaN,No,Está entre mis preferidas,No,166.0,B o 02,19/04,18.181818,100.0,1
4,5,10.0,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,...,NaN,No,La evitaría,No,166.0,B o 02,19/04,36.363636,25.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5077,5253,17.0,15.0,Femenino,Rural,No,Uso compartido con otras personas,Secundaría,Ninguno,Cuida el hogar,...,NaN,No,Me interesa poco,No,36.0,I o 09,02/05,18.181818,100.0,1
5078,5254,24.0,15.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Primaria,Trabaja,...,NaN,No,La evitaría,No,173.0,E o 05,02/05,18.181818,25.0,1
5079,5255,31.0,13.0,Femenino,Rural,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,...,NaN,No,No la conozco,No,36.0,I o 09,02/05,45.454545,25.0,1
5080,5256,5.0,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Secundaría,Trabaja,...,NaN,No,La evitaría,No,6.0,C o 03,02/05,45.454545,50.0,1


In [179]:
df_7['Media nacional resultados'] = df_7['Puntaje conocimiento'].mean()
df_7['Conoce PC'] = 1*(df_7[col_conocePC] > 2)
df_7['Puntaje conocimientos PC'] = 100*(df_7[col_PC].mean(axis=1)/5)
df_7['Puntaje autoeficacia'] = (df_7[col_autoeficacia].mean(axis=1)*100/5)
df_7['Media nacional autoeficacia'] = df_7['Puntaje autoeficacia'].mean()


In [180]:
df_7['Media nacional interés'] = df_7['Puntaje interes'].mean()


In [181]:
cols_dimension7 = ['Puntaje conocimiento', 'Media nacional resultados','Puntaje interes', 'Interesado en tecnologia', 'Conoce PC', 'Puntaje conocimientos PC', 'Puntaje autoeficacia', 'Media nacional autoeficacia', 'Media nacional interés', 'index']
agrupadores_dimension7 = ['mean', 'max', 'mean', 'sum', 'sum', 'mean','mean', 'max', 'max', 'count']
agg_funcs = dict(zip(cols_dimension7, agrupadores_dimension7))

In [213]:
df_institucion = df_7.pivot_table(index=['Código IE'], values=cols_dimension7, aggfunc=agg_funcs).rename(columns={'index':'Cantidad estudiantes'})
df_institucion['% Conoce PC'] = (df_institucion['Conoce PC']/df_institucion['Cantidad estudiantes'])*100
df_institucion['% Interesado'] = (df_institucion['Interesado en tecnologia']/df_institucion['Cantidad estudiantes'])*100

In [214]:
df_institucion['Diferencia resultados'] = (df_institucion['Puntaje conocimiento'] - df_institucion['Media nacional resultados'])*100/df_institucion['Media nacional resultados']

df_institucion['Diferencia Autoeficacia'] = (df_institucion['Puntaje autoeficacia'] - df_institucion['Media nacional autoeficacia'])*100/df_institucion['Media nacional autoeficacia']

df_institucion['Diferencia interes'] = (df_institucion['Puntaje interes'] - df_institucion['Media nacional interés'])*100/df_institucion['Media nacional interés']

In [215]:
df_institucion

,Conoce PC,Interesado en tecnologia,Media nacional autoeficacia,Media nacional interés,Media nacional resultados,Puntaje autoeficacia,Puntaje conocimiento,Puntaje conocimientos PC,Puntaje interes,Cantidad estudiantes,% Conoce PC,% Interesado,Diferencia resultados,Diferencia Autoeficacia,Diferencia interes
Código IE,,,,,,,,,,,,,,,
1.0,70,86,66.61708,40.69756,31.195664,70.137931,30.250784,58.836207,38.146552,116,60.344828,74.137931,-3.028883,5.285208,-6.268209
2.0,35,53,66.61708,40.69756,31.195664,59.104478,24.830393,51.492537,47.388060,67,52.238806,79.104478,-20.404343,-11.277291,16.439560
3.0,41,43,66.61708,40.69756,31.195664,69.904762,29.870130,59.761905,39.285714,63,65.079365,68.253968,-4.249097,4.935194,-3.469116
4.0,1,1,66.61708,40.69756,31.195664,92.000000,18.181818,95.000000,100.000000,1,100.000000,100.000000,-41.716842,38.102721,145.714976
6.0,30,48,66.61708,40.69756,31.195664,63.698113,22.469983,59.905660,55.188679,53,56.603774,90.566038,-27.970814,-4.381709,35.606850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241.0,47,59,66.61708,40.69756,31.195664,68.450000,32.954545,57.625000,38.750000,80,58.750000,73.750000,5.638225,2.751427,-4.785447
247.0,27,28,66.61708,40.69756,31.195664,64.947368,11.244019,64.736842,33.552632,38,71.052632,73.684211,-63.956468,-2.506431,-17.556159
248.0,7,8,66.61708,40.69756,31.195664,67.000000,22.727273,57.916667,33.333333,12,58.333333,66.666667,-27.146052,0.574808,-18.095008


In [236]:
df_institucion['Nivel ConocePC'] =  pd.cut(df_institucion['% Conoce PC'], bins=[0, 20, 40, 50, 60, 100], labels=['1A', '1B', '2A', '2B', '3'], include_lowest=True)

df_institucion['Nivel PuntajePC'] =  pd.cut(df_institucion['Puntaje conocimientos PC'], bins=[0, 50, 60,70, 100], labels=['1A', '1B',"2A",'3'], include_lowest=True)

df_institucion['Nivel Resultados'] = pd.cut(df_institucion['Diferencia resultados'], bins=[-100, -50, -30,  0, 10,20, 100], labels=['1A', '1B', "2A", "2B", '3', '4'], include_lowest=True)

df_institucion['Nivel Interes'] = pd.cut(df_institucion['% Interesado'], bins=[0, 20, 40, 60, 80, 100], labels=['1A', '1B', "2A", "2B", '4'], include_lowest=True)

df_institucion['Nivel Interes pares'] = pd.cut(df_institucion['Diferencia interes'], bins=[-100,-50, -30, 0 , 200], labels=['1B', "2A", "2B", '4'], include_lowest=True)

df_institucion['Nivel Autoeficacia'] = pd.cut(df_institucion['Puntaje autoeficacia'], bins=[0, 30, 60, 100], labels=['1B', "2A", '4'], include_lowest=True)

df_institucion['Nivel Autoeficacia pares'] = pd.cut(df_institucion['Diferencia Autoeficacia'], bins=[-100, 0, 10, 20, 100], labels=['2A',"2B", "3", '4'], include_lowest=True)


df_institucion

,Conoce PC,Interesado en tecnologia,Media nacional autoeficacia,Media nacional interés,Media nacional resultados,Puntaje autoeficacia,Puntaje conocimiento,Puntaje conocimientos PC,Puntaje interes,Cantidad estudiantes,...,Diferencia Autoeficacia,Diferencia interes,Nivel ConocePC,Nivel PuntajePC,Nivel Resultados,Nivel Interes,Nivel Interes pares,Nivel Autoeficacia,Nivel Autoeficacia pares,Dimensión 7
Código IE,,,,,,,,,,,,,,,,,,,,,
1.0,70,86,66.61708,40.69756,31.195664,70.137931,30.250784,58.836207,38.146552,116,...,5.285208,-6.268209,3,1B,2A,2B,2B,4,2B,1B
2.0,35,53,66.61708,40.69756,31.195664,59.104478,24.830393,51.492537,47.388060,67,...,-11.277291,16.439560,2B,1B,2A,2B,4,2A,2A,1B
3.0,41,43,66.61708,40.69756,31.195664,69.904762,29.870130,59.761905,39.285714,63,...,4.935194,-3.469116,3,1B,2A,2B,2B,4,2B,1B
4.0,1,1,66.61708,40.69756,31.195664,92.000000,18.181818,95.000000,100.000000,1,...,38.102721,145.714976,3,3,1B,4,4,4,4,1B
6.0,30,48,66.61708,40.69756,31.195664,63.698113,22.469983,59.905660,55.188679,53,...,-4.381709,35.606850,2B,1B,2A,4,4,4,2A,1B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241.0,47,59,66.61708,40.69756,31.195664,68.450000,32.954545,57.625000,38.750000,80,...,2.751427,-4.785447,2B,1B,2B,2B,2B,4,2B,1B
247.0,27,28,66.61708,40.69756,31.195664,64.947368,11.244019,64.736842,33.552632,38,...,-2.506431,-17.556159,3,2A,1A,2B,2B,4,2A,1A
248.0,7,8,66.61708,40.69756,31.195664,67.000000,22.727273,57.916667,33.333333,12,...,0.574808,-18.095008,2B,1B,2A,2B,2B,4,2B,1B


In [234]:
df_institucion['Dimensión 7'] = df_institucion.filter(regex='^Nivel').min(axis=1)

In [231]:
df_institucion[['Dimensión 7']].to_excel('Dimensión 7.xlsx')

In [235]:
df_institucion['Dimensión 7']

Código IE
1.0      1B
2.0      1B
3.0      1B
4.0      1B
6.0      1B
         ..
241.0    1B
247.0    1A
248.0    1B
252.0    1B
297.0    2A
Name: Dimensión 7, Length: 104, dtype: object